In [74]:
print("hi")

hi


In [75]:
model_card = 'bert-base-uncased'

# Domain-pre-training corpora
dpt_corpus_train = 'data/dapt_train.txt'
dpt_corpus_train_data_selected = 'data/dapt_train_data_selected.txt'  # Optional: If you want to use data selection for DAPT
dpt_corpus_val = 'data/dapt_val.txt'  # Optional: If you want to use a validation set for DAPT

# Fine-tuning corpora
# If there are multiple downstream NLP tasks/corpora, you can concatenate those files together
ft_corpus_train = 'data/ft_train.txt'

# how should the ft_train.txt look like? ans = "Each line in the ft_train.txt file should represent a single training example for the fine-tuning task. The format of each line will depend on the specific NLP task you are working on. Here are some common formats for different tasks:
# 1. Text Classification:
#    Each line contains the text followed by a tab and then the label.
#    Example:
#    ```
#    This is a positive review.    positive
#    This is a negative review.    negative
#    ```
# should it be a csv or txt?
# ans = "The ft_train.txt file can be either a plain text file (.txt) or a CSV file (.csv), depending on your preference and the tools you are using. If you choose to use a CSV file, make sure to properly format it with commas separating the fields and include headers if necessary. For example, a CSV format for text classification might look like this:
# ```
#     text,label
#     This is a positive review.,positive
#     This is a negative review.,negative
#     ```

In [76]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

model = AutoModelForMaskedLM.from_pretrained(model_card)
tokenizer = AutoTokenizer.from_pretrained(model_card)

k:\ner\nerenv\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [77]:
from pathlib import Path


In [78]:
# Load text data into memory
fine_tuning_texts = Path(ft_corpus_train).read_text().splitlines()
training_texts = Path(dpt_corpus_train).read_text().splitlines()



In [79]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def split_long_documents(texts, max_tokens=512):
    """
    Split each document into chunks with <= max_tokens tokens.
    Returns a flat list of documents.
    """
    split_docs = []
    for text in texts:
        tokens = tokenizer.tokenize(text)  # tokenize to WordPiece tokens
        # Split tokens into chunks of max_tokens
        for i in range(0, len(tokens), max_tokens):
            chunk_tokens = tokens[i:i + max_tokens]
            # Convert back to string
            chunk_text = tokenizer.convert_tokens_to_string(chunk_tokens)
            split_docs.append(chunk_text)
    return split_docs


In [80]:
# Assume you already loaded your lines
# training_texts = list of in-domain texts
# fine_tuning_texts = list of fine-tuning texts

training_texts = split_long_documents(training_texts, max_tokens=512)
fine_tuning_texts = split_long_documents(fine_tuning_texts, max_tokens=512)

print("Training corpus length:", len(training_texts))
print("Fine-tuning corpus length:", len(fine_tuning_texts))


Training corpus length: 10000
Fine-tuning corpus length: 10000


In [81]:
from pathlib import Path

from transformers_domain_adaptation import DataSelector


selector = DataSelector(
    keep=0.5,  # TODO Replace with `keep`
    tokenizer=tokenizer,
    # similarity_metrics=['euclidean'],
    similarity_metrics=['cosine'],
    diversity_metrics=[
        "type_token_ratio",
        "entropy",
    ],
)

In [82]:
# # Load text data into memory
# fine_tuning_texts = Path(ft_corpus_train).read_text().splitlines()
# training_texts = Path(dpt_corpus_train).read_text().splitlines()



In [83]:
# fine_tuning_texts

In [84]:
len(training_texts)

10000

In [85]:
len(fine_tuning_texts)

10000

In [86]:
# Fit on fine-tuning corpus
selector.fit(fine_tuning_texts)

# Select relevant documents from in-domain training corpus
selected_corpus = selector.transform(training_texts)

# Save selected corpus to disk under `dpt_corpus_train_data_selected`
Path(dpt_corpus_train_data_selected).write_text('\n'.join(selected_corpus));

Token indices sequence length is longer than the specified maximum sequence length for this model (73387 > 512). Running this sequence through the model will result in indexing errors
computing similarity:   0%|          | 0/1 [00:00<?, ?metric/s]


TypeError: len() of unsized object